<a href="https://colab.research.google.com/github/kerupinen/Compdb_final_repo/blob/main/Final_Project_Rupinen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing and Connecting to Database

In [1]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.1/670.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 17.1 MB/s eta 0:00:00


In [2]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient(host='3.230.203.12',  # host is the hostname for the database
                                port=27018,  # port is the port number that mongo is running on
                                username='compdb_student',  # username for the db
                                password='compdbpass',  # password for the db
                                authSource='admin')  # Since our user only exists for the emse6992 db, we need to specify this

# Setup collections and connections
db = client.twitter
stats_coll = db.twitter_statuses
retweets_coll = db.twitter_retweets
favs_coll = db.twitter_favorites
lists_coll = db.twitter_lists_updated
friends_coll = db.twitter_friends_updated


In [3]:
import psycopg2
import psycopg2.extras
import pandas as pd

# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

#Work

##Tweets per weekday

In [4]:
x = 1
weekday_count_list = []
total = 0
overall_total = 0

# While the weekday is 1-7 (Monday being 1 and Sunday being 7)
while x < 8:
    # Select number of tweets tweeted where weekday = x
    query = f"SELECT count(distinct(statuses)) FROM twitter.statuses WHERE extract(isodow FROM created_date) = '{x}'"
    cursor.execute(query)  # Executing query
    res = cursor.fetchall()
    # Add number of tweets for each weekday with the number indicating weekday to list
    weekday_count_list.append([x, res[0]['count']])
    # Update day + 1
    x += 1

print(weekday_count_list)

[[1, 6618], [2, 6264], [3, 6684], [4, 6647], [5, 6729], [6, 5172], [7, 4717]]


In [5]:
import plotly.express as px

# Plot in a bar chart tweets by weekday (Monday being 1 and Sunday being 7)
weekday_count_fig = px.bar(
    weekday_count_list,
    x=0,
    y=1,
    title="Tweets by weekday",
    labels={'0': 'Day of Week', '1': 'Number of Tweets'},
    range_y=(4000, 7000)
)
weekday_count_fig.show()

##Retweets and Likes per weekday

In [6]:
x = 1
week_day_totals = []

# While the weekday is 1-7 (Monday being 1 and Sunday being 7)
while x < 8:
    total_likes = 0
    total_retweets = 0

    # LIKES:
    # Select favorites_count from each tweet where weekday = x
    query = f"SELECT distinct(status_id), favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '{x}'"
    cursor.execute(query)  # Executing query
    res = cursor.fetchall()
    # For each favorites count, add to total
    for i in res:
        total_likes += i['favorites_count']

    # RETWEETS:
    # Select retweet_count from each tweet where weekday = x
    query = f"SELECT distinct(status_id), retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '{x}'"
    cursor.execute(query)  # Executing query
    res = cursor.fetchall()
    # For each retweet count, add to total for that weekday
    for w in res:
        total_retweets += w['retweet_count']

    # Add weekday, like total, and retweet total per day to list
    week_day_totals.append([x, total_likes, total_retweets])

    # Update weekday + 1
    x += 1
    # Reset totals for next day
    total_likes = 0
    total_retweets = 0

print(week_day_totals)

[[1, 18805170, 8095532], [2, 20967480, 8806093], [3, 22649066, 11023208], [4, 21073340, 11227533], [5, 28259773, 11119761], [6, 20573004, 16762835], [7, 22489203, 8907312]]


In [7]:
retweet_day = []
likes_day = []
x = 1

# Separating list into retweets per day and likes per day
for i in week_day_totals:
    # Taking retweets
    retweet_day.append([x, i[2]])
    # Taking likes
    likes_day.append([x, i[1]])

    x += 1

In [8]:
#Charting retweets by weekday (Monday being 1 and Sunday being 7)
retweet_day_fig = px.bar(
    retweet_day,
    x=0,
    y=1,
    title = "retweets by weekday",
    labels={'0':'Day of Week','1':'Like per Tweet'},
    range_y=(0,20000000)
)
retweet_day_fig.show()

In [9]:
#Charting Likes By Weekday (Monday being 1 and Sunday being 7)
likes_day_fig = px.bar(
    likes_day,
    x=0,
    y=1,
    title = "Likes by weekday",
    labels={'0':'Day of Week','1':'Like per Tweet'},
    range_y=(0,30000000)
)
likes_day_fig.show()

##Retweets per Tweet and Likes per Tweet per weekday

In [10]:
x = 1
avg_retweet_pd = []
avg_likes_pd = []
#for each favorites count, add to total
while x < 8:
    #Take the total RETWEETS of the weekday and / by total count of tweets from that weekday
    average = (week_day_totals[x-1][2])/(weekday_count_list[x-1][1])
    #Adding to a list of retweets per tweet per day
    avg_retweet_pd.append([x,average])
    #Take the total LIKES of the weekday and / by total count of tweets from that weekday
    average = (week_day_totals[x-1][1])/(weekday_count_list[x-1][1])
    #Adding to a list of likes per tweet per day
    avg_likes_pd.append([x,average])
    #Update weekday+1
    x += 1

print(avg_retweet_pd)
print(avg_likes_pd)


[[1, 1223.25959504382], [2, 1405.8258301404853], [3, 1649.1932974266906], [4, 1689.112832856928], [5, 1652.513152028533], [6, 3241.074052590874], [7, 1888.3425906296375]]
[[1, 2841.5185856754306], [2, 3347.2988505747126], [3, 3388.549670855775], [4, 3170.3535429517074], [5, 4199.698766532917], [6, 3977.7656612529004], [7, 4767.691965232139]]


Retweet per tweet

In [11]:
#Chart retweet per tweet by weekday
avg_retweet_fig = px.bar(
    avg_retweet_pd,
    x=0,
    y=1,
    title = "Retweet per Tweet by Weekday",
    labels={'0':'Day of Week','1':'Retweets per Tweet'},
    range_y=(0,4000)
)
avg_retweet_fig.show()

Like Per Tweet

In [12]:
#Charting Likes Per Tweet By Weekday
avg_likes_fig = px.bar(
    avg_likes_pd,
    x=0,
    y=1,
    title = "Likes per Tweet by Weekday",
    labels={'0':'Day of Week','1':'Like per Tweet'},
    range_y=(0,5000)
)
avg_likes_fig.show()

##Tweets per Week of Month

In [13]:
tweets_per_week = []
x = 0
y = 7
week_number = 1

# While the latest day is less than 36
while y < 36:
    # Select distinct number of tweets between (startdate) and (end date) of the week of the month
    sql_query = f"SELECT count(distinct(status_id)) FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(sql_query)  # Executing query
    result = cursor.fetchall()
    # Add number of tweets to list of tweets per week
    tweets_per_week.append([week_number, result[0]['count']])
    # Update x to the next week
    x = y + 1
    # If in the third week, add 10 to catch all values possible in the last week
    if y == 21:
        y = y + 10
    else:
        # Otherwise add 7 to get to the next week
        y = y + 7
    # Update i to show next week
    week_number += 1

print(tweets_per_week)

[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [14]:
#Bar chart of tweets organized by week of the month (1-4)
tweets_per_week_fig = px.bar(
    tweets_per_week,
    x=0,
    y=1,
    title = "Tweets by week",
    labels={'0':'Week of Month','1':'Number of Tweets'},
    range_y=(5000,12000)
)
tweets_per_week_fig.show()

##Retweets and Likes per week of Month

In [15]:
totals_per_week = []
likes_total = 0
retweets_total = 0
x = 0
y = 7
week_number = 1

# While not past the last day of the month
while y < 36:
    # Select tweets likes between (start date) and (end date) of the week of the month
    query_likes = f"SELECT distinct(status_id), favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(query_likes)  # Executing query
    res_likes = cursor.fetchall()

    # Calculate total likes for the week
    for counts in res_likes:
        likes_total += counts['favorites_count']

    # Select tweets retweets between (start date) and (end date) of the week of the month
    query_retweets = f"SELECT distinct(status_id), retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(query_retweets)  # Executing query
    res_retweets = cursor.fetchall()

    # Calculate total retweets for the week
    for retweets in res_retweets:
        retweets_total += retweets['retweet_count']

    # Add both retweets and likes per week totals to a list
    totals_per_week.append([week_number, likes_total, retweets_total])

    # Update week counters and totals
    x = y + 1
    if y == 21:
        y = y + 10
    else:
        y = y + 7
    week_number += 1
    likes_total = 0
    retweets_total = 0

print(totals_per_week)


[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]


In [16]:
retweet_week = []
likes_week = []
x = 1

# For each value in the totals list
for i in totals_per_week:
    # Separate into retweets and likes individual lists
    retweet_week.append([x, i[2]])
    likes_week.append([x, i[1]])
    # Update x
    x += 1

In [17]:
#Chart retweets by week of month
retweet_week_fig = px.bar(
    retweet_week,
    x=0,
    y=1,
    title = "Retweets by Week of Month",
    labels={'0':'Week of Month','1':'Retweet per Tweet'},
    range_y=(0,30000000)
)
retweet_week_fig.show()

In [18]:
#Chart likes by week of month
likes_week_fig = px.bar(
    likes_week,
    x=0,
    y=1,
    title = "Likes by Week of Month",
    labels={'0':'Week of Month','1':'Like per Tweet'},
    range_y=(0,50000000)
)
likes_week_fig.show()

##Retweets per Tweet and Likes per Tweet per Week of Month

In [19]:
print(totals_per_week)
print(tweets_per_week)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]
[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [21]:
x = 1
avg_retweet_pw = []
avg_likes_pw = []

# For each week of the month (1-4)
while x < 5:
    # Take total retweets of the week and divide by total tweets made per week
    retweet_average = totals_per_week[x - 1][2] / tweets_per_week[x - 1][1]
    # Add to retweet per tweet per week list
    avg_retweet_pw.append([x, retweet_average])

    # Take total likes of the week and divide by total tweets made per week
    likes_average = totals_per_week[x - 1][1] / tweets_per_week[x - 1][1]
    # Add to likes per tweet per week list
    avg_likes_pw.append([x, likes_average])

    # Update x
    x += 1

print(avg_retweet_pw)
print(avg_likes_pw)


[[1, 2424.3321765953347], [2, 1458.4412679219208], [3, 1494.1590859865714], [4, 1726.8659510519535]]
[[1, 3566.2895700790436], [2, 2947.621350837796], [3, 3801.8975525232836], [4, 4172.268441391155]]


In [22]:
#Chart retweets per tweet by week of month (1-4)
avg_retweet_pw_fig = px.bar(
    avg_retweet_pw,
    x=0,
    y=1,
    title = "retweet per tweet by week of month",
    labels={'0':'week of month','1':'Retweets per Tweet'},
    range_y=(0,4000))
avg_retweet_pw_fig.show()

In [23]:
#Chart likes per tweet by week of month (1-4)
avg_likes_pw_fig = px.bar(
    avg_likes_pw,
    x=0,
    y=1,
    title = "like per tweet by week of month",
    labels={'0':'week of month','1':'Like per Tweet'},
    range_y=(0,5000)
)
avg_likes_pw_fig.show()

#T-Testing

##Per Weekday

In [24]:
import numpy as np
from scipy import stats

total_other_likes = []
total_day_likes = []
total_other_retweets = []
total_day_retweets = []
results_likes = []
results_retweets = []

# Running for each day of the week
for y in range(1, 8):
    for x in range(1, 8):
        # Select likes where dow = x
        query_likes = f"SELECT distinct(status_id), favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '{x}'"
        cursor.execute(query_likes)
        res_likes = cursor.fetchall()

        for i in res_likes:
            if x != y:
                total_other_likes.append(i['favorites_count'])
            else:
                total_day_likes.append(i['favorites_count'])

        # Select retweets where dow = x
        query_retweets = f"SELECT distinct(status_id), retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '{x}'"
        cursor.execute(query_retweets)
        res_retweets = cursor.fetchall()

        for i in res_retweets:
            if x != y:
                total_other_retweets.append(i['retweet_count'])
            else:
                total_day_retweets.append(i['retweet_count'])

    # Performing t-tests and storing results
    likes_ttest_result = stats.ttest_ind(total_other_likes, total_day_likes)
    retweets_ttest_result = stats.ttest_ind(total_other_retweets, total_day_retweets)

    results_likes.append([y, likes_ttest_result])
    results_retweets.append([y, retweets_ttest_result])

    # Resetting lists
    total_other_likes.clear()
    total_day_likes.clear()
    total_other_retweets.clear()
    total_day_retweets.clear()

print(results_likes)
print(results_retweets)


#running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet/likes counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.

#If the p-value is small (Less than 0.05) and the T-stat is negative,
#it means that it is probable that the likes/retweets average of the singular day is higher than the other days


[[1, TtestResult(statistic=2.6647033197546555, pvalue=0.007708514095540831, df=42829.0)], [2, TtestResult(statistic=0.8919595056465808, pvalue=0.3724196487993565, df=42829.0)], [3, TtestResult(statistic=0.7837005184302288, pvalue=0.43322019212565066, df=42829.0)], [4, TtestResult(statistic=1.5351399731603927, pvalue=0.1247568360203621, df=42829.0)], [5, TtestResult(statistic=-2.0366349872579903, pvalue=0.04169277141028242, df=42829.0)], [6, TtestResult(statistic=-1.0850702927188118, pvalue=0.2778966311258495, df=42829.0)], [7, TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)]]
[[1, TtestResult(statistic=1.9098304005343465, pvalue=0.05616172668248298, df=42829.0)], [2, TtestResult(statistic=1.2350236637963525, pvalue=0.21682847237631958, df=42829.0)], [3, TtestResult(statistic=0.4328120825221578, pvalue=0.6651534742038784, df=42829.0)], [4, TtestResult(statistic=0.2923721947806585, pvalue=0.7700034814982757, df=42829.0)], [5, TtestResult(statistic=0.4

##Per Week of Month

In [25]:
mtotal_other_likes = []
mtotal_week_likes = []
mtotal_other_retweets = []
mtotal_week_retweets = []

x = 0
y = 7

# Running while not past the last week
while y < 36:
    # Selecting likes where week is (1-4)
    query_likes = f"SELECT distinct(status_id), favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(query_likes)
    res_likes = cursor.fetchall()

    for counts in res_likes:
        if x != 15:  # Change 22 to the desired week number
            mtotal_other_likes.append(counts['favorites_count'])
        else:
            mtotal_week_likes.append(counts['favorites_count'])

    # Selecting retweets where week is (1-4)
    query_retweets = f"SELECT distinct(status_id), retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(query_retweets)
    res_retweets = cursor.fetchall()

    for num in res_retweets:
        if x != 0:  # Change 22 to the desired week number
            mtotal_other_retweets.append(num['retweet_count'])
        else:
            mtotal_week_retweets.append(num['retweet_count'])

    # Update week
    x = y + 1
    if y == 21:
        y = y + 10
    else:
        y = y + 7

# Perform t-tests
likes_ttest_result = stats.ttest_ind(mtotal_other_likes, mtotal_week_likes)
retweets_ttest_result = stats.ttest_ind(mtotal_other_retweets, mtotal_week_retweets)

print(likes_ttest_result)
print(retweets_ttest_result)

#running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet/likes counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.

#If the p-value is small (Less than 0.05) and the T-stat is negative,
#it means that it is probable that the likes/retweets average of the singular week is higher than the other weeks


TtestResult(statistic=-0.7916432527561208, pvalue=0.42857310357108114, df=42829.0)
TtestResult(statistic=-2.9919571090785113, pvalue=0.0027735304052264693, df=42829.0)


#When does Elon Musk Tweet?

In [26]:
import plotly.express as px

weekday_count_elon = []

# Going through all 7 days (1 Monday to 7 Sunday)
for x in range(1, 8):
    # Select tweets made by Elon Musk on the current day
    query = f"SELECT count(distinct(statuses)) FROM twitter.statuses WHERE user_id = '44196397' AND extract(isodow FROM created_date) = '{x}'"
    cursor.execute(query)  # Executing query
    res = cursor.fetchall()
    # Add to day and count list
    weekday_count_elon.append([x, res[0]['count']])

print(weekday_count_elon)

# Chart Elon Tweets by Weekday
fig = px.bar(weekday_count_elon, x=0, y=1, title="Tweets by weekday", labels={'0': 'Day of Week', '1': 'Number of Tweets'})
fig.show()

[[1, 76], [2, 70], [3, 77], [4, 98], [5, 72], [6, 81], [7, 97]]


In [27]:
import plotly.express as px

x = 1
week_count_elon = []
total_elon = 0
ototal_elon = 0
y = 7
num = 0

# Going through all four weeks
while y < 36:
    # Select Elon's count of tweets from (start date) to (end date)
    query = f"SELECT count(distinct(statuses)) FROM twitter.statuses WHERE user_id = '44196397' AND extract(day FROM created_date) BETWEEN '{x}' and '{y}'"
    cursor.execute(query)  # Executing query
    res = cursor.fetchall()
    # Add to day and count list
    week_count_elon.append([num, res[0]['count']])
    # Update dates
    x = y + 1
    # If third week add 10 to get all dates to catch all final dates
    if y == 21:
        y = y + 10
    # Otherwise add 7 to go to next week
    else:
        y = y + 7
    # Update week number
    num = num + 1

print(week_count_elon)

# Chart Elon's Tweets by Week
fig = px.bar(week_count_elon, x=0, y=1, title="Elon's Tweets by week of month", labels={'0': 'Week of Month', '1': 'Number of Tweets'})
fig.show()


[[0, 151], [1, 105], [2, 133], [3, 182]]


#Putting It All Together

In [31]:
# Selecting for likes from best week (last week) and best dow (5)
query_best_week = "SELECT favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '5' AND extract(day FROM created_date) BETWEEN '22' and '32'"
cursor.execute(query_best_week)  # Executing query
res_best_week = cursor.fetchall()
best_tweets = []
# Add likes to best tweets list
for x in res_best_week:
    best_tweets.append(x['favorites_count'])

# Selecting for likes NOT from best week (last week) and best dow (5)
query_other_weeks = "SELECT favorites_count FROM twitter.statuses WHERE extract (isodow FROM created_date) != '5' AND extract(day FROM created_date) NOT BETWEEN '22' and '32'"
cursor.execute(query_other_weeks)  # Executing query
res_other_weeks = cursor.fetchall()
other_tweets = []
# Add likes to other tweets list
for x in res_other_weeks:
    other_tweets.append(x['favorites_count'])

print(str(stats.ttest_ind(other_tweets, best_tweets)))


TtestResult(statistic=-1.0131971728341616, pvalue=0.31097452228879296, df=28733.0)


In [29]:
# Selecting for retweets from best week (first week) and best dow (5)
query_best_week = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '6' AND extract(day FROM created_date) BETWEEN '1' and '7'"
cursor.execute(query_best_week)  # Executing query
res_best_week = cursor.fetchall()
best_tweets = []
# Add retweets to best tweets list
for x in res_best_week:
    best_tweets.append(x['retweet_count'])

# Selecting for retweets NOT from best week (first week) and best dow (5)
query_other_weeks = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) != '6' AND extract(day FROM created_date) NOT BETWEEN '1' and '7'"
cursor.execute(query_other_weeks)  # Executing query
res_other_weeks = cursor.fetchall()
other_tweets = []
# Add retweets to other tweets list
for x in res_other_weeks:
    other_tweets.append(x['retweet_count'])

print(str(stats.ttest_ind(other_tweets, best_tweets)))

TtestResult(statistic=-5.168866635358705, pvalue=2.3703694891158435e-07, df=29787.0)
